In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import random
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
import struct
from struct import unpack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 7.5 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.4.0-py3-none-any.whl size=88775 sha256=8249f1c20e587e09e2b7ea6d5c24a84054231c7686e84c32155f245b43fb2a85
  Stored in directory: /root/.cache/pip/wheels/7a/2b/da/aec872f95d2c24105496ef149a9a576f52daf686f8f2127541
Successfully built cairocffi


In [ ]:
# Env vars
torch.use_deterministic_algorithms(False)

# Const vars
LOAD_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN baseline 256 no pool.pt'
RAND_SEED = 0
DEVICE = "cuda"

IMG_SIDE = 256
NUM_CLASSES = 25
EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 128
LOSS_FN = nn.CrossEntropyLoss()

In [ ]:
# convert raw vector image to single raster image
def vector_to_raster(vector_image, side=IMG_SIDE, line_diameter=16, padding=80, bg_color=(0,0,0), fg_color=(1,1,1)):
  """
  padding and line_diameter are relative to the original 256x256 image.
  """
  
  original_side = 256.
  
  surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
  ctx = cairo.Context(surface)
  ctx.set_antialias(cairo.ANTIALIAS_BEST)
  ctx.set_line_cap(cairo.LINE_CAP_ROUND)
  ctx.set_line_join(cairo.LINE_JOIN_ROUND)
  ctx.set_line_width(line_diameter)

  # scale to match the new size
  # add padding at the edges for the line_diameter
  # and add additional padding to account for antialiasing
  total_padding = padding * 2. + line_diameter
  new_scale = float(side) / float(original_side + total_padding)
  ctx.scale(new_scale, new_scale)
  ctx.translate(total_padding / 2., total_padding / 2.)
      
  bbox = np.hstack(vector_image).max(axis=1)
  offset = ((original_side, original_side) - bbox) / 2.
  offset = offset.reshape(-1,1)
  centered = [stroke + offset for stroke in vector_image]

  # clear background
  ctx.set_source_rgb(*bg_color)
  ctx.paint()

  # draw strokes, this is the most cpu-intensive part
  ctx.set_source_rgb(*fg_color)     
  for xv, yv in centered:   
    ctx.move_to(xv[0], yv[0])
    for x, y in zip(xv, yv):
        ctx.line_to(x, y)
    ctx.stroke()

  data = surface.get_data()
  raster = np.copy(np.asarray(data)[::4]).reshape(side, side)
  return raster

# Define transformation(s) to be applied to dataset-
transforms_norm = T.Compose(
      [
          T.ToTensor(), # scales integer inputs in the range [0, 255] into the range [0.0, 1.0]
          T.Normalize(mean=(0.138), std=(0.296)) # Quickdraw mean and stdev (35.213, 75.588), divided by 255
      ]
  )

# transform functions - take sketch image, return torch tensor of descriptors
def transform(vector_img, is_test):
  raster = vector_to_raster(vector_img)
  raster = transforms_norm(raster)

  # add rotations and translations at test time
  if is_test: 
    angle = random.random()*60 - 30
    deltaX = random.randint(-10, 10)
    deltaY = random.randint(-10, 10)

    raster = T.functional.affine(raster, angle, [deltaX, deltaY], 1, 0,
                                 interpolation=T.InterpolationMode.BILINEAR)
  return raster

# helper method to find class based on imgset index
def find_class(idx, num_list):
  class_id = 0
  sum = num_list[class_id]
  while idx >= sum:
    class_id += 1
    sum += num_list[class_id]
  return class_id

# deterministic worker re-seeding
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# custom dataset for quickdraw
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, nums, is_test):
    self.imgs = imgs
    self.nums = nums
    self.len = sum(nums)
    self.is_test = is_test

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx]
    x = transform(img, self.is_test)
    y = find_class(idx, self.nums)
    return x, y


# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 64, 3)
#         self.conv2 = nn.Conv2d(64, 64, 3)
#         self.conv3 = nn.Conv2d(64, 64, 3)
#         self.conv4 = nn.Conv2d(64, 64, 3)
#         self.maxpool = nn.MaxPool2d(2) 
#         self.relu = nn.ReLU()
#         self.fc1 = nn.Linear(64 * 4 * 4, 384)
#         self.head = nn.Linear(384, NUM_CLASSES)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu(x)
#         x = self.conv2(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.conv3(x)
#         x = self.relu(x)
#         x = self.conv4(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = self.relu(x)
#         return self.head(x)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(64, 128, 3, stride=2, padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3, stride=2, padding=1)
        self.maxpool = nn.MaxPool2d(2) 
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.head = nn.Linear(512, NUM_CLASSES)

    def forward(self, x):
        x = self.conv1(x)
        # x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv2(x)
        # x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv3(x)
        # x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv4(x)
        # x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv5(x)
        # x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv6(x)
        # x = self.maxpool(x)
        x = self.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        return self.head(x)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(DEVICE), y.to(DEVICE)
      # Compute prediction and loss
      out = model(x)
      loss = loss_fn(out, y)

      # Backpropagation to update model parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print current training metrics for user
      y, out, loss = y.to("cpu"), out.to("cpu"), loss.to("cpu")
      loss_val = loss.item()
      if batch % 50 == 0:
          current = (batch + 1) * BATCH_SIZE
          print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

      pred = out.argmax(dim=1, keepdim=True)
      correct = pred.eq(y.view_as(pred)).sum().item()
      total_correct += correct
      total_loss += loss_val
      # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{(size // BATCH_SIZE) + 1:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / ((size // BATCH_SIZE) + 1):>7f}   epoch avg train accuracy: {total_correct / size:>7f}")
      
def eval_loop(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    accuracy = total_correct / size
    return accuracy


In [ ]:
# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image


def unpack_drawings(filename):
  imageset = []
  with open(filename, 'rb') as f:
      while True:
          try:
              imageset.append(unpack_drawing(f))
          except struct.error:
              break
  return imageset

train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Test/'
train_imgs = []
test_imgs = []
train_nums = []
test_nums = []
list_of_classes = ["arm", "brain", "circle", "ear", "elbow", "eye", 
                   "face", "finger", "foot", "hand", "hexagon", "knee", 
                   "leg", "line", "mouth", "nose", "octagon", 
                   'skull', 'square', 'squiggle', 'star', 'toe', 'tooth', 
                   'triangle', 'zigzag']
# list_of_classes = ["aircraft carrier", "airplane", "alarm clock", "ambulance", 
#                    "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
#                    "backpack", "banana", "bandage", "barn", "baseball bat", 
#                    "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
#                    "beard", "bed", "bee"]

In [ ]:
# load dataset
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  test_folder = test_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings
  train_nums.append(len(train_drawings))
  test_drawings = unpack_drawings(test_folder)
  test_imgs += test_drawings
  test_nums.append(len(test_drawings))

In [ ]:
DEVICE = "cpu"
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)
g = torch.Generator()
g.manual_seed(RAND_SEED)

def get_confusion_mat(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  conf_mat = np.zeros((NUM_CLASSES, NUM_CLASSES))
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      y = y.view_as(pred)
      total_correct += pred.eq(y).sum().item()
      y, pred = y.numpy(), pred.numpy()
      indices = np.concatenate((y, pred), axis=1)
      for i in range(indices.shape[0]):
        conf_mat[indices[i, 0], indices[i, 1]] += 1

    accuracy = total_correct / size
    print(f"test accuracy: {accuracy:>7f}")
    return conf_mat

eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

model = CNN()
checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)

random.seed(RAND_SEED)
conf_mat = get_confusion_mat(dataloader=eval_loader,model=model)
conf_mat_norm = conf_mat * (255 / 100)
cv2_imshow(conf_mat_norm)

diag = np.diagonal(conf_mat)
sums = np.asarray(test_nums)
accuracies = diag / sums
acc_sorted_idxs = np.argsort(accuracies)
for i in range(NUM_CLASSES):
  print(list_of_classes[acc_sorted_idxs[i]] + " accuracy " + str(accuracies[acc_sorted_idxs[i]]))

print("\n")

diag_mat = np.multiply(np.identity(NUM_CLASSES), conf_mat)
conf_mat_nodiag = conf_mat - diag_mat
conf_mat_sorted = np.argsort(conf_mat_nodiag, axis=None)
for i in range(1, 6):
  print("class " + list_of_classes[conf_mat_sorted[-i] // NUM_CLASSES] + " misclassified as " + list_of_classes[conf_mat_sorted[-i] % NUM_CLASSES])

test accuracy: 0.767107


octagon accuracy 0.44
toe accuracy 0.5858585858585859
squiggle accuracy 0.6
foot accuracy 0.62
leg accuracy 0.64
nose accuracy 0.64
knee accuracy 0.66
arm accuracy 0.67
finger accuracy 0.69
zigzag accuracy 0.73
elbow accuracy 0.74
brain accuracy 0.77
ear accuracy 0.79
tooth accuracy 0.8
hexagon accuracy 0.83
eye accuracy 0.84
skull accuracy 0.86
star accuracy 0.88
hand accuracy 0.9
triangle accuracy 0.9
face accuracy 0.9
mouth accuracy 0.91
line accuracy 0.92
circle accuracy 0.92
square accuracy 0.94


class arm misclassified as square
class arm misclassified as circle
class arm misclassified as line
class arm misclassified as mouth
class arm misclassified as face


In [ ]:
diag_mat = np.multiply(np.identity(NUM_CLASSES), conf_mat)
conf_mat_nodiag = conf_mat - diag_mat
conf_mat_sorted = np.argsort(conf_mat_nodiag, axis=None)
for i in range(1, 6):
  print("class " + list_of_classes[conf_mat_sorted[-i] // NUM_CLASSES] + " misclassified as " + list_of_classes[conf_mat_sorted[-i] % NUM_CLASSES])

class octagon misclassified as hexagon
class zigzag misclassified as squiggle
class finger misclassified as toe
class knee misclassified as leg
class leg misclassified as foot


In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN baseline 256 no pool.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.215159  [  128/25000]
loss: 1.965218  [ 6528/25000]
loss: 1.599877  [12928/25000]
loss: 1.217440  [19328/25000]

epoch avg train loss: 1.649858   epoch avg train accuracy: 0.510040

-------------------------------

Epoch 2

loss: 1.275178  [  128/25000]
loss: 1.112569  [ 6528/25000]
loss: 0.964037  [12928/25000]
loss: 1.005727  [19328/25000]

epoch avg train loss: 1.028137   epoch avg train accuracy: 0.691840

-------------------------------

Epoch 3

loss: 0.882377  [  128/25000]
loss: 0.770213  [ 6528/25000]
loss: 0.759027  [12928/25000]
loss: 0.788708  [19328/25000]

epoch avg train loss: 0.820373   epoch avg train accuracy: 0.751880

-------------------------------

Epoch 4

loss: 0.725505  [  128/25000]
loss: 0.627760  [ 6528/25000]
loss: 0.802505  [12928/25000]
loss: 0.809654  [19328/25000]

epoch avg train loss: 0.663724   epoch avg train accuracy: 0.793160

-------------------------------

Epoch 5

loss: 0.518464  [  128/25000]
loss: 0.445447  [ 6528/25000]
los

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN baseline 28.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.218138  [  512/25000]
loss: 2.311960  [ 5632/25000]
loss: 1.955709  [10752/25000]
loss: 1.801892  [15872/25000]
loss: 1.498721  [20992/25000]

epoch avg train loss: 2.031463   epoch avg train accuracy: 0.408520

-------------------------------

Epoch 2

loss: 1.512664  [  512/25000]
loss: 1.488144  [ 5632/25000]
loss: 1.255664  [10752/25000]
loss: 1.250665  [15872/25000]
loss: 1.268072  [20992/25000]

epoch avg train loss: 1.297988   epoch avg train accuracy: 0.617600

-------------------------------

Epoch 3

loss: 1.146265  [  512/25000]
loss: 1.072307  [ 5632/25000]
loss: 1.054770  [10752/25000]
loss: 1.045690  [15872/25000]
loss: 1.003103  [20992/25000]

epoch avg train loss: 1.033741   epoch avg train accuracy: 0.693160

-------------------------------

Epoch 4

loss: 0.902011  [  512/25000]
loss: 0.860329  [ 5632/25000]
loss: 1.002645  [10752/25000]
loss: 0.954934  [15872/25000]
loss: 0.970173  [20992/25000]

epoch avg train loss: 0.899375   epoch avg train accur

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25-2 CNN baseline 256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.221095  [  128/24999]
loss: 2.173836  [ 6528/24999]
loss: 1.980121  [12928/24999]
loss: 1.215533  [19328/24999]

epoch avg train loss: 1.816887   epoch avg train accuracy: 0.468459

-------------------------------

Epoch 2

loss: 1.287434  [  128/24999]
loss: 0.953429  [ 6528/24999]
loss: 1.114062  [12928/24999]
loss: 0.919442  [19328/24999]

epoch avg train loss: 0.914766   epoch avg train accuracy: 0.738870

-------------------------------

Epoch 3

loss: 0.699980  [  128/24999]
loss: 0.740001  [ 6528/24999]
loss: 0.690159  [12928/24999]
loss: 0.638084  [19328/24999]

epoch avg train loss: 0.698257   epoch avg train accuracy: 0.798592

-------------------------------

Epoch 4

loss: 0.531516  [  128/24999]
loss: 0.732716  [ 6528/24999]
loss: 0.555673  [12928/24999]
loss: 0.611803  [19328/24999]

epoch avg train loss: 0.572231   epoch avg train accuracy: 0.831873

-------------------------------

Epoch 5

loss: 0.529996  [  128/24999]
loss: 0.353951  [ 6528/24999]
los

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN baseline 256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.215283  [  128/25000]
loss: 1.917566  [ 6528/25000]
loss: 1.432460  [12928/25000]
loss: 1.097957  [19328/25000]

epoch avg train loss: 1.550189   epoch avg train accuracy: 0.535440

-------------------------------

Epoch 2

loss: 1.091581  [  128/25000]
loss: 1.002626  [ 6528/25000]
loss: 0.751829  [12928/25000]
loss: 0.809524  [19328/25000]

epoch avg train loss: 0.888813   epoch avg train accuracy: 0.734640

-------------------------------

Epoch 3

loss: 0.794531  [  128/25000]
loss: 0.715321  [ 6528/25000]
loss: 0.647214  [12928/25000]
loss: 0.691696  [19328/25000]

epoch avg train loss: 0.718833   epoch avg train accuracy: 0.779600

-------------------------------

Epoch 4

loss: 0.587544  [  128/25000]
loss: 0.600346  [ 6528/25000]
loss: 0.669575  [12928/25000]
loss: 0.814611  [19328/25000]

epoch avg train loss: 0.601495   epoch avg train accuracy: 0.813400

-------------------------------

Epoch 5

loss: 0.573084  [  128/25000]
loss: 0.484653  [ 6528/25000]
los

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optim.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/CNN subset256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 11

loss: 0.085136  [  128/28000]
loss: 0.225925  [ 6528/28000]
loss: 0.222469  [12928/28000]
loss: 0.159301  [19328/28000]
loss: 0.082144  [25728/28000]

epoch avg train loss: 0.166116   epoch avg train accuracy: 0.942071

-------------------------------

Epoch 12

loss: 0.151190  [  128/28000]
loss: 0.050427  [ 6528/28000]
loss: 0.145639  [12928/28000]
loss: 0.176640  [19328/28000]
loss: 0.117802  [25728/28000]

epoch avg train loss: 0.146608   epoch avg train accuracy: 0.947500

-------------------------------

Epoch 13

loss: 0.126263  [  128/28000]
loss: 0.045405  [ 6528/28000]
loss: 0.076877  [12928/28000]
loss: 0.099668  [19328/28000]
loss: 0.097276  [25728/28000]

epoch avg train loss: 0.130560   epoch avg train accuracy: 0.954786

-------------------------------

Epoch 14

loss: 0.115557  [  128/28000]
loss: 0.092650  [ 6528/28000]
loss: 0.104343  [12928/28000]
loss: 0.079545  [19328/28000]
loss: 0.192434  [25728/28000]

epoch avg train loss: 0.097053   epoch avg train a

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/CNN subset256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
for i in range(30):
  eval_loop(dataloader=test_loader,model=model)

Epoch 1

loss: 3.333196  [  128/28000]
loss: 1.414870  [12928/28000]
loss: 0.924474  [25728/28000]

epoch avg train loss: 1.608316   epoch avg train accuracy: 0.530107

-------------------------------

Epoch 2

loss: 1.053097  [  128/28000]
loss: 1.012632  [12928/28000]
loss: 1.032461  [25728/28000]

epoch avg train loss: 0.937371   epoch avg train accuracy: 0.723857

-------------------------------

Epoch 3

loss: 0.630781  [  128/28000]
loss: 0.908870  [12928/28000]
loss: 0.695954  [25728/28000]

epoch avg train loss: 0.761911   epoch avg train accuracy: 0.769179

-------------------------------

Epoch 4

loss: 0.649218  [  128/28000]
loss: 0.463790  [12928/28000]
loss: 0.612333  [25728/28000]

epoch avg train loss: 0.643948   epoch avg train accuracy: 0.800107

-------------------------------

Epoch 5

loss: 0.562698  [  128/28000]
loss: 0.766773  [12928/28000]
loss: 0.628773  [25728/28000]

epoch avg train loss: 0.558278   epoch avg train accuracy: 0.822857

-----------------------

KeyboardInterrupt: ignored

In [ ]:
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

0.6389067524115755
0.007904832500237147
